# Anwendung 1: Keyword Extraction

Eine mögliche Anwendung von Textanalysen auf der Grundlage eines Vector-Space-Modells ist die Identifikation von Schlüsselwörtern für Texte, die »Keyword Extraction«. Ein Schlüsselwort kann verstanden werden als ein Wort, dass einen inhaltlichen Aspekt eines Textes beschreibt. Eine begrenzte Anzahl von Schlüsselwörtern sollte also bereits einen guten Eindruck vom Inhalt eines Textes vermitteln. Schlagwortsysteme, etwa in Bibliothekskatalogen oder Literaturdatenbanken, bauen auf dieser Idee auf. Sie erfordern aber eine sorgfältige manuelle Verschlagwortung. Daher ist die Frage interessant, ob sich Schlüsselwörter für Texte auch automatisch aus dem Textinhalt erschließen lassen.

Ein erster Kandidat für die Identifikation wäre die reine Worthäufigkeit: Für einen Text relevante Wörter sollten in ihm häufig vorkommen. Die Häufigkeitsauszählungen aus den vorangegangenen Einheiten zeigen aber, dass eine reine Häufigkeitsliste nicht sehr hilfreich ist: Zunächst erscheinen Funktionwörter wie etwa Artikel ganz oben in der Liste. Aber auch wenn man nach Wortarten filtert, bleiben oft relativ unspezifische Wörter dominant.

Der Keyword Extraction liegt daher die Annahme zugrunde, dass einige Wörter insgesamt häufig vorkommen, ohne dass sie inhaltlich aussagekräftig wären. Ihr häufiges Vorkommen ist daher eher charakteristisch für eine (Fach-)Sprache als für einzelne Texte. Die Verteilung dieser Wörter sollte entsprechend über die Texte relativ gleich bleiben. Ein Schlüsselwort dagegen sollte im entsprechenden Text deutlich häufiger vorkommen als im Gesamtkorpus. Statistisch lässt sich ein Schlüsselwort also definieren als ein Wort, das in einem einzelnen Text signifikant häufiger vorkommt als seine Häufigkeit im Gesamtcorpus erwarten lassen würde.

Eine Form einer entsprechenden Identifikation von Schlüsselwörtern ist [Tf-Idf](http://de.wikipedia.org/wiki/Tf-idf-Ma%C3%9F). Das Kürzel steht dabei für »Termfrequenz – Inverse Dokumentfrequenz«.

Dieses Verfahren ist in gensim implementiert. Dafür wird die in der vorangegangenen Einheit vorgestellte Corpus-Klasse verwendet.

In [1]:
from gensim.corpora.textcorpus import TextCorpus
from textblob_de import TextBlobDE as TextBlob
import pandas as pd

class CSVCorpus(TextCorpus):
    """Read corpus from a csv file."""

    def get_texts(self):
        with self.getstream() as csvfile:  # Öffnet die CSV-Datei
            table = pd.read_csv(csvfile, parse_dates=['date'], encoding='utf-8')  # Liest die CSV-Datei
            for text in table['text']:  # Verarbeite die einzelnen Texte aus der Spalte 'text'
                blob = TextBlob(text)
                yield blob.words

Das Vorgehen in gensim ist dabei nicht ganz selbsterklären, daher hier eine kurze Erläuterung: Der Befehl `TfidfModel(corpus)` erzeugt zunächst ein Modell, das die allgemeine Häufigkeitsverteilung von Wörtern im Corpus enthält. Mit diesem Modell können nun einzelne Dokumente, aber auch nur einzelne Wörter aus einem Dokument, verglichen werden. Dies ist etwa dann sinnvoll, wenn eine Berechnung für das gesamte Corpus zu aufwändig wäre, oder wenn neue Texte hinzukommen, die mit einem bestehenden Corpus verglichen werden sollen.

Um dagegen für das gesamte Corpus die Tf-Idf-Werte zu berechnen, wird dem Modell im zweiten Schritt einfach wiederum das gesamte Corpus übergeben: `tf_idf[corpus]`. In diesem Schritt findet dann die eigentliche Berechnung für die einzelnen Dokument statt.

In [2]:
from gensim.models import TfidfModel

corpus = CSVCorpus("../Daten/Reden.csv")
tf_idf = TfidfModel(corpus)
corpus_idf = tf_idf[corpus]

Dies gibt für jedes Wort in jedem Dokument einen Wert aus. Das Format ist dabei das gleiche wie für Corpora, nur dass anstelle der reinen Worthäufigkeiten der tfidf-Wert ausgegeben wird. Ein Dokument ist also eine Liste aus `(wortindex, wert)`-Paaren.

Um nun für einzelne Dokumente die Schlüsselwörter zu bestimmen, ist es sinnvoll, die Wörter im Dokument nach ihrem tfidf-Wert zu sortieren. Zugleich interessieren nicht die numerischen Wort-Indices, sondern die eigentlichen Wörter. Daher wird hier eine Funktion `keywords` definiert, die das Dokument zunächst nach tfidf-Wert (dem zweiten Eintrag im Wort-Wert-Paar: `x[1]`) sortiert und dann ein Paar aus Wörterbucheintrag `corpus.dictionary[term]` und tfidf-Wert ausgibt.

In [3]:
def keywords(corpus, doc_tf_idf):
    return [(corpus.dictionary[term], value)
            for term, value
            in sorted(doc_tf_idf, key=lambda x: x[1], reverse=True)]

Für die ersten zehn Dokumente können nun etwa jeweils fünf Schlüsselwörter ausgegeben werden:

In [4]:
for i, doc in enumerate(corpus_idf):
    if i == 9:
        break
    print()
    for keyword, value in keywords(corpus, doc)[0:5]:
        print('  {} ({})'.format(keyword, value))


  forscht“ (0.43699409511465326)
  „Jugend (0.43699409511465326)
  Entwicklungsland (0.1334158004695894)
  stachelt (0.1334158004695894)
  Gymnasium (0.11956330874423221)

  Frauen (0.5199507857411889)
  Geschichten (0.1356255710995334)
  unermesslichem (0.13245827136131796)
  G7-Dialogforum (0.13245827136131796)
  Jinping (0.13245827136131796)

  WHO (0.2877199964065426)
  Krankheit (0.22439674978794663)
  Informationen (0.2226976416575381)
  medizinischen (0.17771860955768617)
  Solberg (0.1745585137750599)

  Sustainable (0.19723335271907866)
  Goals (0.18386621698688743)
  Development (0.16702561760653403)
  Wirtschaft (0.1652491508498979)
  Korruption (0.13552487703988408)

  Ziele (0.2563022568576125)
  Sustainable (0.17806274139533296)
  Generalsekretär (0.16899992422066198)
  Goals (0.16599485936491587)
  Development (0.150791126065927)

  2030 (0.2077824618151842)
  globale (0.14397981111432034)
  Agenda (0.1416998433562269)
  mobilisieren (0.1258821139298492)
  regional (0.1

Hier ist auffällig, dass fast nur Substantive ausgegeben werden, obwohl das Corpus selbst die Wörter überhaupt nicht gefiltert hat. Dies legt den Schluss nahe, dass andere Wortarten deutlich gleicher über das Corpus verteilt sind.

Um die Plausibilität dieser Listen zu überprüfen, können zusätzlich die Titel der Dokumente ausgegeben werden:

In [5]:
data = pd.read_csv("../Daten/Reden.csv", parse_dates=['date'], encoding='utf-8')

for i, doc in enumerate(corpus_idf):
    if i == 9:
        break
    print()
    print(data['title'][i])
    for keyword, value in keywords(corpus, doc)[0:5]:
        print('  {} ({})'.format(keyword, value))


Rede von Bundeskanzlerin Merkel beim Empfang der Preisträgerinnen und Preisträger des 50. Bundeswettbewerbs „Jugend forscht“ am 30. September 2015
  forscht“ (0.43699409511465326)
  „Jugend (0.43699409511465326)
  Entwicklungsland (0.1334158004695894)
  stachelt (0.1334158004695894)
  Gymnasium (0.11956330874423221)

Rede von Bundeskanzlerin Merkel beim Global Leaders‘ Meeting on Achieving Gender Equality and Women’s Empowerment: „A Commitment to Action“ am 27. September 2015
  Frauen (0.5199507857411889)
  Geschichten (0.1356255710995334)
  unermesslichem (0.13245827136131796)
  G7-Dialogforum (0.13245827136131796)
  Jinping (0.13245827136131796)

Rede von Bundeskanzlerin Merkel beim gemeinsamen Side Event der Bundesrepublik Deutschland, des Königreichs Norwegen und der Republik Ghana „Securing a healthy future“ am 26. September 2015
  WHO (0.2877199964065426)
  Krankheit (0.22439674978794663)
  Informationen (0.2226976416575381)
  medizinischen (0.17771860955768617)
  Solberg (0.174